## 1. Data Preparation

### R kernel:

In [3]:
# load requirements
library(dplyr)
library(Seurat)
library(SeuratObject)
library(Matrix)
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/CellPhoneDB process.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/NATMI process.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/run_CellPhoneDB.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/run_NATMI.R")

In [2]:
# load single-cell sample data
sampleA <- readRDS("https://zenodo.org/record/6497091/files/GSE122960.rds?download=1")
sampleB <- readRDS("https://zenodo.org/record/6497091/files/GSE128033.rds?download=1")
sampleC <- readRDS("https://zenodo.org/record/6497091/files/GSE135893.rds?download=1")
sampleD <- readRDS("https://zenodo.org/record/6497091/files/GSE136831.rds?download=1")

In [5]:
# normalized data of the sample
str(sampleA@assays$RNA@data)
str(sampleB@assays$RNA@data)
str(sampleC@assays$RNA@data)
str(sampleD@assays$RNA@data)

Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:15028219] 33 35 45 54 55 59 61 72 86 97 ...
  ..@ p       : int [1:10414] 0 3337 5097 7069 10281 13289 15511 18031 19740 21390 ...
  ..@ Dim     : int [1:2] 33694 10413
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:33694] "RP11-34P13.3" "FAM138A" "OR4F5" "RP11-34P13.7" ...
  .. ..$ : chr [1:10413] "AAACCTGAGCACCGCT-1_2" "AAACCTGCACAGATTC-1_2" "AAACCTGGTAAGGGCT-1_2" "AAACCTGGTCAAACTC-1_2" ...
  ..@ x       : num [1:15028219] 0.686 0.401 1.09 0.401 0.686 ...
  ..@ factors : list()
Formal class 'dgCMatrix' [package "Matrix"] with 6 slots
  ..@ i       : int [1:39789428] 47 59 86 97 153 176 189 197 216 219 ...
  ..@ p       : int [1:20628] 0 1379 2847 7202 8373 9651 13150 14990 16263 18922 ...
  ..@ Dim     : int [1:2] 45068 20627
  ..@ Dimnames:List of 2
  .. ..$ : chr [1:45068] "RP11-34P13.3" "FAM138A" "OR4F5" "RP11-34P13.7" ...
  .. ..$ : chr [1:20627] "AAACCTGAGAGGACGG-1_2" "AAACCTGAGCGTCAAG-1_2" "AAACCTGCAGGA

In [5]:
# cell types and amounts of the sample
table(Idents(sampleA))
table(Idents(sampleB))
table(Idents(sampleC))
table(Idents(sampleD))


 Macrophage  Fibroblast         AT2    Monocyte       Tcell         AT1 
       5892         147        2293         774         645         210 
Endothelial        Mast 
        299         153 


Endothelial         AT2  Macrophage    Monocyte  Fibroblast       Tcell 
       2553        1548        7622        1153        3420        1693 
       Mast         AT1 
       1405        1233 


        AT2         AT1 Endothelial        Mast    Monocyte       Tcell 
       3225          96        3496         360        2113        2829 
 Macrophage  Fibroblast 
      16507         152 


 Macrophage    Monocyte       Tcell        Mast  Fibroblast         AT2 
      60483        6793        5088         572        1443         496 
Endothelial         AT1 
       3379         176 

In [4]:
# load gold standard sample
gold <- read.table("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/data/IPF gold standard.txt", header = TRUE, sep = "\t")
str(gold)
head(gold)

'data.frame':	208 obs. of  4 variables:
 $ source  : chr  "AT1" "AT1" "AT1" "AT1" ...
 $ target  : chr  "Monocyte" "Monocyte" "Monocyte" "Macrophage" ...
 $ ligand  : chr  "TGFB1" "TGFB1" "TGFB1" "TGFB1" ...
 $ receptor: chr  "TGFBR1" "TGFBR2" "TGFBR3" "TGFBR1" ...


,source,target,ligand,receptor
,<chr>,<chr>,<chr>,<chr>
1,AT1,Monocyte,TGFB1,TGFBR1
2,AT1,Monocyte,TGFB1,TGFBR2
3,AT1,Monocyte,TGFB1,TGFBR3
4,AT1,Macrophage,TGFB1,TGFBR1
5,AT1,Macrophage,TGFB1,TGFBR2
6,AT1,Macrophage,TGFB1,TGFBR3


In [5]:
# remove unwanted genes for prediction
genes <- unique(c(gold$ligand, gold$receptor))
sampleA <- subset(sampleA, features = genes) 
sampleB <- subset(sampleB, features = genes)
sampleC <- subset(sampleC, features = genes) 
sampleD <- subset(sampleD, features = genes) 

## 2. Cell-cell Interaction Prediction 

In [6]:
# CellPhoneDB
CellPhoneDB_process(sampleA)
CellPhoneDB_process(sampleB)
CellPhoneDB_process(sampleC)
CellPhoneDB_process(sampleD)

### Python kernel:

In [1]:
!ls -AR cpdb

cpdb:
sampleA  sampleB  sampleC  sampleD

cpdb/sampleA:
barcodes.tsv  features.tsv  matrix.mtx	meta.tsv

cpdb/sampleB:
barcodes.tsv  features.tsv  matrix.mtx	meta.tsv

cpdb/sampleC:
barcodes.tsv  features.tsv  matrix.mtx	meta.tsv

cpdb/sampleD:
barcodes.tsv  features.tsv  matrix.mtx	meta.tsv


In [2]:
!source activate cpdb

In [2]:
!cellphonedb method statistical_analysis cpdb/sampleA/meta.tsv cpdb/sampleA/ --counts-data hgnc_symbol --output-path cpdb/sampleA/out

[ ][APP][25/04/22-16:52:37][WARNING] Latest local available version is `v2.0.0`, using it
[ ][APP][25/04/22-16:52:37][WARNING] User selected downloaded database `v2.0.0` is available, using it
[ ][CORE][25/04/22-16:52:37][INFO] Initializing SqlAlchemy CellPhoneDB Core
[ ][CORE][25/04/22-16:52:37][INFO] Using custom database at /home/bioinformatics/.cpdb/releases/v2.0.0/cellphone.db
[ ][APP][25/04/22-16:52:37][INFO] Launching Method cpdb_statistical_analysis_local_method_launcher
[ ][APP][25/04/22-16:52:37][INFO] Launching Method _set_paths
[ ][APP][25/04/22-16:52:37][INFO] Launching Method _load_meta_counts
[ ][APP][25/04/22-16:52:37][INFO] Launching Method _check_counts_data
[ ][CORE][25/04/22-16:52:37][INFO] Launching Method cpdb_statistical_analysis_launcher
[ ][CORE][25/04/22-16:52:37][INFO] Launching Method _counts_validations
[ ][CORE][25/04/22-16:52:37][INFO] Launching Method get_interactions_genes_complex
[ ][CORE][25/04/22-16:52:38][INFO] [Cluster Statistical Analysis] Thresho

In [3]:
!cellphonedb method statistical_analysis cpdb/sampleB/meta.tsv cpdb/sampleB/ --counts-data hgnc_symbol --output-path cpdb/sampleB/out

[ ][APP][25/04/22-17:34:45][WARNING] Latest local available version is `v2.0.0`, using it
[ ][APP][25/04/22-17:34:45][WARNING] User selected downloaded database `v2.0.0` is available, using it
[ ][CORE][25/04/22-17:34:45][INFO] Initializing SqlAlchemy CellPhoneDB Core
[ ][CORE][25/04/22-17:34:45][INFO] Using custom database at /home/bioinformatics/.cpdb/releases/v2.0.0/cellphone.db
[ ][APP][25/04/22-17:34:45][INFO] Launching Method cpdb_statistical_analysis_local_method_launcher
[ ][APP][25/04/22-17:34:45][INFO] Launching Method _set_paths
[ ][APP][25/04/22-17:34:45][INFO] Launching Method _load_meta_counts
[ ][APP][25/04/22-17:34:45][INFO] Launching Method _check_counts_data
[ ][CORE][25/04/22-17:34:45][INFO] Launching Method cpdb_statistical_analysis_launcher
[ ][CORE][25/04/22-17:34:45][INFO] Launching Method _counts_validations
[ ][CORE][25/04/22-17:34:45][INFO] Launching Method get_interactions_genes_complex
[ ][CORE][25/04/22-17:34:46][INFO] [Cluster Statistical Analysis] Thresho

In [4]:
!cellphonedb method statistical_analysis cpdb/sampleC/meta.tsv cpdb/sampleC/ --counts-data hgnc_symbol --output-path cpdb/sampleC/out

[ ][APP][25/04/22-18:55:33][WARNING] Latest local available version is `v2.0.0`, using it
[ ][APP][25/04/22-18:55:33][WARNING] User selected downloaded database `v2.0.0` is available, using it
[ ][CORE][25/04/22-18:55:33][INFO] Initializing SqlAlchemy CellPhoneDB Core
[ ][CORE][25/04/22-18:55:33][INFO] Using custom database at /home/bioinformatics/.cpdb/releases/v2.0.0/cellphone.db
[ ][APP][25/04/22-18:55:33][INFO] Launching Method cpdb_statistical_analysis_local_method_launcher
[ ][APP][25/04/22-18:55:33][INFO] Launching Method _set_paths
[ ][APP][25/04/22-18:55:33][INFO] Launching Method _load_meta_counts
[ ][APP][25/04/22-18:55:33][INFO] Launching Method _check_counts_data
[ ][CORE][25/04/22-18:55:33][INFO] Launching Method cpdb_statistical_analysis_launcher
[ ][CORE][25/04/22-18:55:33][INFO] Launching Method _counts_validations
[ ][CORE][25/04/22-18:55:34][INFO] Launching Method get_interactions_genes_complex
[ ][CORE][25/04/22-18:55:34][INFO] [Cluster Statistical Analysis] Thresho

In [5]:
!cellphonedb method statistical_analysis cpdb/sampleD/meta.tsv cpdb/sampleD/ --counts-data hgnc_symbol --output-path cpdb/sampleD/out

[ ][APP][25/04/22-20:48:22][WARNING] Latest local available version is `v2.0.0`, using it
[ ][APP][25/04/22-20:48:22][WARNING] User selected downloaded database `v2.0.0` is available, using it
[ ][CORE][25/04/22-20:48:22][INFO] Initializing SqlAlchemy CellPhoneDB Core
[ ][CORE][25/04/22-20:48:22][INFO] Using custom database at /home/bioinformatics/.cpdb/releases/v2.0.0/cellphone.db
[ ][APP][25/04/22-20:48:22][INFO] Launching Method cpdb_statistical_analysis_local_method_launcher
[ ][APP][25/04/22-20:48:22][INFO] Launching Method _set_paths
[ ][APP][25/04/22-20:48:22][INFO] Launching Method _load_meta_counts
[ ][APP][25/04/22-20:48:23][INFO] Launching Method _check_counts_data
[ ][CORE][25/04/22-20:48:23][INFO] Launching Method cpdb_statistical_analysis_launcher
[ ][CORE][25/04/22-20:48:23][INFO] Launching Method _counts_validations
[ ][CORE][25/04/22-20:48:23][INFO] Launching Method get_interactions_genes_complex
[ ][CORE][25/04/22-20:48:23][INFO] [Cluster Statistical Analysis] Thresho

### R kernel:

In [12]:
CellPhoneDBA <- run_CellPhoneDB_output("sampleA")
head(CellPhoneDBA)

,cellA,cellB,geneA,geneB
,<chr>,<chr>,<chr>,<chr>
1,AT1,Endothelial,TGFB1,TGFBR3
2,AT1,Fibroblast,PDGFA,PDGFRA
3,Endothelial,Endothelial,TGFB1,TGFBR3
4,Endothelial,Fibroblast,PDGFB,PDGFRA
5,Endothelial,Fibroblast,PDGFB,PDGFRB
6,Endothelial,Fibroblast,PDGFB,LRP1


In [13]:
CellPhoneDBB <- run_CellPhoneDB_output("sampleB")
head(CellPhoneDBB)

,cellA,cellB,geneA,geneB
,<chr>,<chr>,<chr>,<chr>
1,AT2,Endothelial,TGFB1,TGFBR3
2,AT2,Fibroblast,PDGFA,PDGFRA
3,Endothelial,Endothelial,TGFB1,TGFBR3
4,Fibroblast,AT2,IGF1,IGF1R
5,Fibroblast,AT2,PDGFRA,PDGFC
6,Fibroblast,Endothelial,TGFB1,TGFBR3


In [14]:
CellPhoneDBC <- run_CellPhoneDB_output("sampleC")
head(CellPhoneDBC)

,cellA,cellB,geneA,geneB
,<chr>,<chr>,<chr>,<chr>
1,AT1,Fibroblast,PDGFA,PDGFRA
2,AT2,Endothelial,TGFB1,TGFBR3
3,Endothelial,Endothelial,TGFB1,TGFBR3
4,Endothelial,Fibroblast,PDGFB,PDGFRA
5,Endothelial,Fibroblast,PDGFB,PDGFRB
6,Endothelial,Fibroblast,PDGFB,LRP1


In [15]:
CellPhoneDBD <- run_CellPhoneDB_output("sampleD")
head(CellPhoneDBD)

,cellA,cellB,geneA,geneB
,<chr>,<chr>,<chr>,<chr>
1,AT1,Endothelial,TGFB1,TGFBR3
2,AT1,Fibroblast,TGFB1,TGFBR3
3,AT2,Endothelial,TGFB1,TGFBR3
4,AT2,Fibroblast,TGFB1,TGFBR3
5,Endothelial,Endothelial,TGFB1,TGFBR3
6,Endothelial,Fibroblast,PDGFB,PDGFRB


In [16]:
# NATMI
NATMI_process(sampleA)
NATMI_process(sampleB)
NATMI_process(sampleC)
NATMI_process(sampleD)

### Python kernel:

In [1]:
!ls -AR natmi

natmi:
sampleA  sampleB  sampleC  sampleD

natmi/sampleA:
ann.csv  em.csv

natmi/sampleB:
ann.csv  em.csv

natmi/sampleC:
ann.csv  em.csv

natmi/sampleD:
ann.csv  em.csv


In [2]:
!docker run --rm -it --name natmi -v /home/bioinformatics/Documents/Benchmark/Python/natmi/sampleA/:/opt/NATMI/workdir asrhou/natmi python ExtractEdges.py --emFile /opt/NATMI/workdir/em.csv --annFile /opt/NATMI/workdir/ann.csv --interDB lrc2p --out /opt/NATMI/workdir/out

Input data:
Species to obtain abundance data: human
The expression matrix file: /opt/NATMI/workdir/em.csv
The metafile: /opt/NATMI/workdir/ann.csv
The ligand-receptor interaction database: lrc2p.csv
Species to obtain ligand-receptor interactions: human
The number of cores to use: 1
The folder to save all results: /opt/NATMI/workdir/out
#### the folder "/opt/NATMI/workdir/out" has been created to save the analysis results
#### cluster the expression matrix
#### generate cell-to-cell interactions
#### extract signaling factors
#### construct signaling interactions
#### collected 1512 LR-mediated edges
#### DONE


In [3]:
!docker run --rm -it --name natmi -v /home/bioinformatics/Documents/Benchmark/Python/natmi/sampleA/:/opt/NATMI/workdir asrhou/natmi python VisInteractions.py --sourceFolder /opt/NATMI/workdir/out --interDB lrc2p --weightType mean --detectionThreshold 0.2

Input data:
The source dataset folder: /opt/NATMI/workdir/out
The ligand-receptor interaction database: lrc2p
The weight type of cell-to-cell signaling: mean
The detection threshold for interactions to draw is: 0.2
The specicificity threshold for interactions to draw is: 0
The expression threshold for interactions to draw is: 0
The network layout: kk
The font size for cluster labels: 8
The edge width: 0
The maximum radius of the clusters: 0
The relative distance between clusters: 1
The plot width: 12
The plot height: 10
The plot format: pdf
#### start to construct the cell-to-cell communication network
#### 51 edges are loaded
#### plotting the weighted directed cell-to-cell communication network
#### the folder "/opt/NATMI/workdir/out/Network_exp_0_spe_0_det_0.2_top_0_signal_lrc2p_weight_mean" has been created to save the analysis results
#### DONE


In [4]:
!docker run --rm -it --name natmi -v /home/bioinformatics/Documents/Benchmark/Python/natmi/sampleB/:/opt/NATMI/workdir asrhou/natmi python ExtractEdges.py --emFile /opt/NATMI/workdir/em.csv --annFile /opt/NATMI/workdir/ann.csv --interDB lrc2p --out /opt/NATMI/workdir/out

Input data:
Species to obtain abundance data: human
The expression matrix file: /opt/NATMI/workdir/em.csv
The metafile: /opt/NATMI/workdir/ann.csv
The ligand-receptor interaction database: lrc2p.csv
Species to obtain ligand-receptor interactions: human
The number of cores to use: 1
The folder to save all results: /opt/NATMI/workdir/out
#### the folder "/opt/NATMI/workdir/out" has been created to save the analysis results
#### cluster the expression matrix
#### generate cell-to-cell interactions
#### extract signaling factors
#### construct signaling interactions
#### collected 1856 LR-mediated edges
#### DONE


In [7]:
!docker run --rm -it --name natmi -v /home/bioinformatics/Documents/Benchmark/Python/natmi/sampleB/:/opt/NATMI/workdir asrhou/natmi python VisInteractions.py --sourceFolder /opt/NATMI/workdir/out --interDB lrc2p --weightType mean --detectionThreshold 0.2

Input data:
The source dataset folder: /opt/NATMI/workdir/out
The ligand-receptor interaction database: lrc2p
The weight type of cell-to-cell signaling: mean
The detection threshold for interactions to draw is: 0.2
The specicificity threshold for interactions to draw is: 0
The expression threshold for interactions to draw is: 0
The network layout: kk
The font size for cluster labels: 8
The edge width: 0
The maximum radius of the clusters: 0
The relative distance between clusters: 1
The plot width: 12
The plot height: 10
The plot format: pdf
#### start to construct the cell-to-cell communication network
#### 85 edges are loaded
#### plotting the weighted directed cell-to-cell communication network
#### the folder "/opt/NATMI/workdir/out/Network_exp_0_spe_0_det_0.2_top_0_signal_lrc2p_weight_mean" has been created to save the analysis results
#### DONE


In [5]:
!docker run --rm -it --name natmi -v /home/bioinformatics/Documents/Benchmark/Python/natmi/sampleC/:/opt/NATMI/workdir asrhou/natmi python ExtractEdges.py --emFile /opt/NATMI/workdir/em.csv --annFile /opt/NATMI/workdir/ann.csv --interDB lrc2p --out /opt/NATMI/workdir/out

Input data:
Species to obtain abundance data: human
The expression matrix file: /opt/NATMI/workdir/em.csv
The metafile: /opt/NATMI/workdir/ann.csv
The ligand-receptor interaction database: lrc2p.csv
Species to obtain ligand-receptor interactions: human
The number of cores to use: 1
The folder to save all results: /opt/NATMI/workdir/out
#### the folder "/opt/NATMI/workdir/out" has been created to save the analysis results
#### cluster the expression matrix
#### generate cell-to-cell interactions
#### extract signaling factors
#### construct signaling interactions
#### collected 1521 LR-mediated edges
#### DONE


In [8]:
!docker run --rm -it --name natmi -v /home/bioinformatics/Documents/Benchmark/Python/natmi/sampleC/:/opt/NATMI/workdir asrhou/natmi python VisInteractions.py --sourceFolder /opt/NATMI/workdir/out --interDB lrc2p --weightType mean --detectionThreshold 0.2

Input data:
The source dataset folder: /opt/NATMI/workdir/out
The ligand-receptor interaction database: lrc2p
The weight type of cell-to-cell signaling: mean
The detection threshold for interactions to draw is: 0.2
The specicificity threshold for interactions to draw is: 0
The expression threshold for interactions to draw is: 0
The network layout: kk
The font size for cluster labels: 8
The edge width: 0
The maximum radius of the clusters: 0
The relative distance between clusters: 1
The plot width: 12
The plot height: 10
The plot format: pdf
#### start to construct the cell-to-cell communication network
#### 132 edges are loaded
#### plotting the weighted directed cell-to-cell communication network
#### the folder "/opt/NATMI/workdir/out/Network_exp_0_spe_0_det_0.2_top_0_signal_lrc2p_weight_mean" has been created to save the analysis results
#### DONE


In [6]:
!docker run --rm -it --name natmi -v /home/bioinformatics/Documents/Benchmark/Python/natmi/sampleD/:/opt/NATMI/workdir asrhou/natmi python ExtractEdges.py --emFile /opt/NATMI/workdir/em.csv --annFile /opt/NATMI/workdir/ann.csv --interDB lrc2p --out /opt/NATMI/workdir/out

Input data:
Species to obtain abundance data: human
The expression matrix file: /opt/NATMI/workdir/em.csv
The metafile: /opt/NATMI/workdir/ann.csv
The ligand-receptor interaction database: lrc2p.csv
Species to obtain ligand-receptor interactions: human
The number of cores to use: 1
The folder to save all results: /opt/NATMI/workdir/out
#### the folder "/opt/NATMI/workdir/out" has been created to save the analysis results
#### cluster the expression matrix
#### generate cell-to-cell interactions
#### extract signaling factors
#### construct signaling interactions
#### collected 1674 LR-mediated edges
#### DONE


In [9]:
!docker run --rm -it --name natmi -v /home/bioinformatics/Documents/Benchmark/Python/natmi/sampleD/:/opt/NATMI/workdir asrhou/natmi python VisInteractions.py --sourceFolder /opt/NATMI/workdir/out --interDB lrc2p --weightType mean --detectionThreshold 0.2

Input data:
The source dataset folder: /opt/NATMI/workdir/out
The ligand-receptor interaction database: lrc2p
The weight type of cell-to-cell signaling: mean
The detection threshold for interactions to draw is: 0.2
The specicificity threshold for interactions to draw is: 0
The expression threshold for interactions to draw is: 0
The network layout: kk
The font size for cluster labels: 8
The edge width: 0
The maximum radius of the clusters: 0
The relative distance between clusters: 1
The plot width: 12
The plot height: 10
The plot format: pdf
#### start to construct the cell-to-cell communication network
#### 172 edges are loaded
#### plotting the weighted directed cell-to-cell communication network
#### the folder "/opt/NATMI/workdir/out/Network_exp_0_spe_0_det_0.2_top_0_signal_lrc2p_weight_mean" has been created to save the analysis results
#### DONE


### R kernel:

In [23]:
NATMIA <- run_NATMI_output("sampleA")
head(NATMIA)

,source,target,ligand,receptor
,<chr>,<chr>,<chr>,<chr>
1,AT1,Macrophage,CCL2,CCR1
2,AT1,Monocyte,CCL2,CCR1
3,Fibroblast,Macrophage,CCL2,CCR1
4,Fibroblast,Monocyte,CCL2,CCR1
5,Macrophage,Macrophage,CCL2,CCR1
6,Macrophage,Monocyte,CCL2,CCR1


In [24]:
NATMIB <- run_NATMI_output("sampleB")
head(NATMIB)

,source,target,ligand,receptor
,<chr>,<chr>,<chr>,<chr>
1,Endothelial,Macrophage,CCL2,CCR1
2,Fibroblast,Macrophage,CCL2,CCR1
3,Macrophage,Macrophage,CCL2,CCR1
4,Monocyte,Macrophage,CCL2,CCR1
5,Macrophage,Macrophage,CCL3,CCR1
6,Monocyte,Macrophage,CCL3,CCR1


In [25]:
NATMIC <- run_NATMI_output("sampleC")
head(NATMIC)

,source,target,ligand,receptor
,<chr>,<chr>,<chr>,<chr>
1,Endothelial,Macrophage,CCL2,CCR1
2,Endothelial,Monocyte,CCL2,CCR1
3,Fibroblast,Macrophage,CCL2,CCR1
4,Fibroblast,Monocyte,CCL2,CCR1
5,Macrophage,Macrophage,CCL2,CCR1
6,Macrophage,Monocyte,CCL2,CCR1


In [26]:
NATMID <- run_NATMI_output("sampleD")
head(NATMID)

,source,target,ligand,receptor
,<chr>,<chr>,<chr>,<chr>
1,AT1,Macrophage,CCL2,CCR1
2,Endothelial,Macrophage,CCL2,CCR1
3,Fibroblast,Macrophage,CCL2,CCR1
4,Macrophage,Macrophage,CCL2,CCR1
5,Tcell,Macrophage,CCL4,CCR1
6,Fibroblast,Macrophage,CSF1,CSF1R


## 3. SSP Calculation

In [17]:
# load functions
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/CellPhoneDB_SSP_calculation.R")
source("https://raw.githubusercontent.com/mora-lab/cell-cell-interactions/main/benchmark-workflow/R/NATMI_SSP_calculation.R")

In [18]:
CPDBSSPA <- run_CellPhoneDB_SSP(CellPhoneDBA,sampleA,gold)
CPDBSSPB <- run_CellPhoneDB_SSP(CellPhoneDBB,sampleB,gold)
CPDBSSPC <- run_CellPhoneDB_SSP(CellPhoneDBC,sampleC,gold)
CPDBSSPD <- run_CellPhoneDB_SSP(CellPhoneDBD,sampleD,gold)

In [27]:
NATMISSPA <- run_NATMI_SSP(NATMIA,sampleA,gold)
NATMISSPB <- run_NATMI_SSP(NATMIB,sampleB,gold)
NATMISSPC <- run_NATMI_SSP(NATMIC,sampleC,gold)
NATMISSPD <- run_NATMI_SSP(NATMID,sampleD,gold)

In [21]:
cbind(CPDBSSPA,CPDBSSPB,CPDBSSPC,CPDBSSPD)

CPDBSSPA  CPDBSSPB   CPDBSSPC   CPDBSSPD 
Accuracy    0.3793103 0.2291667  0.25       0.4177215
Sensitivity 0.1195652 0.05978261 0.06989247 0.1823204
Specificity 0.9829868 0.9824894  0.981569   0.9781991

In [28]:
cbind(NATMISSPA,NATMISSPB,NATMISSPC,NATMISSPD)

NATMISSPA NATMISSPB NATMISSPC NATMISSPD
Accuracy    0.3333333 0.2705882 0.280303  0.3895349
Sensitivity 0.0923913 0.125     0.201087  0.3641304
Specificity 0.983463  0.9698444 0.9537938 0.94893